### How to compute the annotation for Haziq's code

Place this file in the cloned JAAD github from here: https://github.com/ykotseruba/JAAD
then run the jupyter notebook

In [1]:

from jaad_data import JAAD
import pandas as pd
import os
import copy
jaad_path = "../JAAD"
jaad = JAAD(data_path=jaad_path)

In [2]:
output_folder = "luc_data_again/"
if not os.path.exists(output_folder):
        os.mkdir(output_folder)

In [3]:
def append_crossing_meta_data(data):
    """
    Appends crossing meta data to each pedestrian. It adds "will_cross" and "crossing_at" attributes.
    will_cross: 1 if the pedestrian if the pedestrian crosses at one point in the sequence 0 if not
    crossing_at: -1 if he does not cross or the frame at which the pedestrian crosses
    
    Args:
        data (dict): JAAD dataset dict generated from generate_database()

    Returns:
        dict: JAAD dataset dict generated from generate_database() augmented with crossing meta data
    
    """
    for video in data.keys():
        video_dict = data[video]["ped_annotations"]
        for pedestrian in video_dict.keys():
            ped_data = video_dict[pedestrian]
            old_id = ped_data["old_id"]
            # get type
            is_pedestrian = "pedestrian" in old_id
            if not is_pedestrian: # will not cross
                ped_data["will_cross"] = 0
                ped_data["crossing_at"] = -1
            else: # append crossing frame if it exists

                cross_states = ped_data["behavior"]["cross"]
                if sum(cross_states) == 0:
                    ped_data["will_cross"] = 0
                    ped_data["crossing_at"] = -1
                else:
                    crossing_point = cross_states.index(1) # finds index of first 1
                    ped_data["will_cross"] = 1
                    ped_data["crossing_at"] = ped_data["frames"][crossing_point]
    return data
                      

In [4]:
data = jaad.generate_database()

---------------------------------------------------------
Generating database for jaad
jaad database loaded from D:\Program Files\EPFL\Project\JAAD\data_cache\jaad_database.pkl


In [5]:
data = append_crossing_meta_data(data)

In [6]:
test = data['video_0305']['ped_annotations']
for p in test.keys():
    print(test[p]['will_cross'])

1
1
0
1
0
0
0
0
0
1
0
0
1
0


In [7]:
def create_video_frame_pedestrian_df(data):
    """
    Creates dataframe with information of each (video,frame,unique_pedestrian) triplet 
    
    Args:
        data (dict): JAAD dataset dict generated from generate_database()

    Returns:
        dataframe: dataframe with information of each (video,frame,unique_pedestrian) triplet 
    
    """
    video_frame_pedestrian_to_info_map = dict()

    for video in data.keys():
        video_dict = data[video]["ped_annotations"]
        for pedestrian in video_dict.keys():
            
            ped_frames = video_dict[pedestrian]["frames"]
            for frame in ped_frames:
                # create key
                key = ",".join([video, str(f"{frame:05d}"), pedestrian])
                
                if video_frame_pedestrian_to_info_map.get(key):
                    print("bug")
                # add to map
                video_frame_pedestrian_to_info_map[key] = get_data(data, key)
               
    return pd.DataFrame.from_dict(video_frame_pedestrian_to_info_map, orient='index')

def get_data(data, key):
    """
    Creates dict with information of each (video,frame,unique_pedestrian) triplet in the key
    info has attributes:
    'video_str', 'frame_str', 'video', 'frame', 'im_w', 'im_h', 'ID',
       'unique_ID', 'type', 'x', 'y', 'w', 'h', 'occlusion', 'walking',
       'standing', 'looking', 'standing_pred', 'walking_pred', 'crossing_pred',
       'crossing_true', 'scenefolderpath', 'imagefolderpath', 'filename',
       'will_cross', 'crossing_at'
    
    Args:
        data (dict): JAAD dataset dict generated from generate_database()
        key (str): key in format "(video,frame,ped)"

    Returns:
        dataframe: dict with information of each (video,frame,unique_pedestrian) triplet 
    
    """
    video, frame, ped = key.split(",")
    frame_int = int(frame)
    
    video_data = data[video]
    ped_data = video_data["ped_annotations"][ped]
    video_frame_pedestrian_info = dict()
    
    # add video in string format
    video_frame_pedestrian_info["video_str"] = video
    
    # add frame in string format
    video_frame_pedestrian_info["frame_str"] = frame
    
    # add video in integer format
    video_frame_pedestrian_info["video"] = int(video[-4:])
    
    # add frame in integer format
    video_frame_pedestrian_info["frame"] = frame_int
    
    # add im_w and im_h
    video_frame_pedestrian_info["im_w"] = video_data["width"]
    video_frame_pedestrian_info["im_h"] = video_data["height"]

    # add IDs
    old_id = ped_data["old_id"]
    video_frame_pedestrian_info["ID"] = old_id
    video_frame_pedestrian_info["unique_ID"] = ped
    
    # get type
    ped_type = "pedestrian" if "pedestrian" in old_id else "ped"
    is_pedestrian = ped_type == "pedestrian"
    video_frame_pedestrian_info["type"] = ped_type

    # get index in list
    index = ped_data["frames"].index(frame_int)

    # add bounding box
    bbox = ped_data["bbox"][index]
    video_frame_pedestrian_info["x"] = bbox[0]
    video_frame_pedestrian_info["y"] = bbox[1]
    video_frame_pedestrian_info["w"] = (bbox[2] - bbox[0])
    video_frame_pedestrian_info["h"] = (bbox[3] - bbox[1])

    # add occlusion
    video_frame_pedestrian_info["occlusion"] = ped_data["occlusion"][index]

    # add walking

    video_frame_pedestrian_info["walking"] = ped_data["behavior"]["action"][index] if is_pedestrian else 0
    # add standing
    video_frame_pedestrian_info["standing"] = int(not ped_data["behavior"]["action"][index]) if is_pedestrian else 0
    # add looking
    video_frame_pedestrian_info["looking"] = ped_data["behavior"]["look"][index] if is_pedestrian else 0

    # add standing_pred
    video_frame_pedestrian_info["standing_pred"] = -1
    # add walking_pred
    video_frame_pedestrian_info["walking_pred"] = -1
    # add crossing_pred
    video_frame_pedestrian_info["crossing_pred"] = -1

    # add crossing
    if ped_type == "ped": # ped types don't impact the driver
        video_frame_pedestrian_info["crossing_true"] = 0
    else:
        video_frame_pedestrian_info["crossing_true"] = ped_data["behavior"]["cross"][index]
        
    # add image folder path
    path =  "/".join(["/work", "vita", "datasets", "JAAD", "images", video])
    video_frame_pedestrian_info["scenefolderpath"] = path

    # add scenefolder path
    video_frame_pedestrian_info["imagefolderpath"] = "huehue"

    # add filename
    video_frame_pedestrian_info["filename"] = frame + ".png"
    
    # add will_cross
    video_frame_pedestrian_info["will_cross"] = ped_data["will_cross"]

    # add cross_at
    video_frame_pedestrian_info["crossing_at"] = ped_data["crossing_at"]
    
    return video_frame_pedestrian_info



In [8]:
df_vdf = create_video_frame_pedestrian_df(data)
df_vdf.head()

,video_str,frame_str,video,frame,im_w,im_h,ID,unique_ID,type,x,...,looking,standing_pred,walking_pred,crossing_pred,crossing_true,scenefolderpath,imagefolderpath,filename,will_cross,crossing_at
"video_0001,00000,0_1_3b",video_0001,00000,1,0,1920,1080,pedestrian1,0_1_3b,pedestrian,465.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0001,huehue,00000.png,0,-1
"video_0001,00001,0_1_3b",video_0001,00001,1,1,1920,1080,pedestrian1,0_1_3b,pedestrian,463.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0001,huehue,00001.png,0,-1
"video_0001,00002,0_1_3b",video_0001,00002,1,2,1920,1080,pedestrian1,0_1_3b,pedestrian,461.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0001,huehue,00002.png,0,-1
"video_0001,00003,0_1_3b",video_0001,00003,1,3,1920,1080,pedestrian1,0_1_3b,pedestrian,459.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0001,huehue,00003.png,0,-1
"video_0001,00004,0_1_3b",video_0001,00004,1,4,1920,1080,pedestrian1,0_1_3b,pedestrian,458.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0001,huehue,00004.png,0,-1


In [9]:
df_vdf.columns

Index(['video_str', 'frame_str', 'video', 'frame', 'im_w', 'im_h', 'ID',
       'unique_ID', 'type', 'x', 'y', 'w', 'h', 'occlusion', 'walking',
       'standing', 'looking', 'standing_pred', 'walking_pred', 'crossing_pred',
       'crossing_true', 'scenefolderpath', 'imagefolderpath', 'filename',
       'will_cross', 'crossing_at'],
      dtype='object')

In [10]:
def get_video_frame_df(df_vdf):
    """Returns grouped dataframe by video and frame aggregating the list of pedestrian within that pair and their informations"""
    return df_vdf.groupby(['frame','video'], as_index=False).agg(list)
        

In [11]:
def save_dataframe(df_train, df_val, time=0,suffix=""):
    """Saves dataframes"""
    train_dir = output_folder + "singletxt_train_"+ str(time) + "s" + suffix
    val_dir = output_folder + "singletxt_val_"+ str(time) + "s" + suffix
    
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)
    if not os.path.exists(val_dir):
        os.mkdir(val_dir)
    
    train_name = "/train"
    val_name = "/val"
    if "truncated" in suffix:
        train_name = train_name + "truncated"
        val_name = val_name + "truncated"
        
    df_train.drop(columns=["frame","video"]).to_csv(train_dir + train_name+".txt", index=False)
    df_val.drop(columns=["frame","video"]).to_csv(val_dir + val_name+".txt", index=False)

In [12]:
def split_dataframe(df, at_video=300):
    """Split dataframes in train and validation dataframes"""
    filter_train = df["video"] <= at_video
    df_train = df[filter_train]
    df_val = df[~filter_train]

    return df_train, df_val

In [13]:
def remove_occlusions(df_vdf, threshold=0):
    """ Remove any instances where the pedestrian is occluded above the allowed threshold (0=0%, 1=25%-75%, 2=75%+)"""
    return df_vdf[df_vdf["occlusion"] <= threshold]

In [14]:
def invert(df_vdf, inverted_frames):
    """Inverts crossing frames "inverted_frames" before crossing so the label is 1"""
    df_copy = df_vdf.copy()
    if inverted_frames == 0:
        return df_vdf
    
    if inverted_frames < 0: # infinity
        df_copy.loc[(df_vdf["will_cross"] == 1) & (df_vdf["frame"] <= df_vdf["crossing_at"]), "crossing_true"] = 1
    else:
        df_copy.loc[(df_vdf["will_cross"] == 1) & (df_vdf["frame"] <= df_vdf["crossing_at"]) & (df_vdf["frame"] >= df_vdf["crossing_at"] - inverted_frames),"crossing_true"] = 1
    
    return df_copy

    

In [15]:
def truncate(df_vdf):
    """Truncates all sequences after crossing. Any pedestrian that does not cross has his full sequence left intact"""
    return df_vdf.loc[(df_vdf["frame"] <= df_vdf["crossing_at"])].append(df_vdf.loc[df_vdf["will_cross"] == 0])


In [16]:
t = truncate(df_vdf[df_vdf["video"] > 300])
t = t[t["will_cross"] == 1]

t.groupby(['unique_ID'], as_index=False).agg(list)

,unique_ID,video_str,frame_str,video,frame,im_w,im_h,ID,type,x,...,looking,standing_pred,walking_pred,crossing_pred,crossing_true,scenefolderpath,imagefolderpath,filename,will_cross,crossing_at
0,0_301_2333b,"[video_0301, video_0301, video_0301, video_030...","[00005, 00006, 00007, 00008, 00009, 00010, 000...","[301, 301, 301, 301, 301, 301, 301, 301, 301, ...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[1920, 1920, 1920, 1920, 1920, 1920, 1920, 192...","[1080, 1080, 1080, 1080, 1080, 1080, 1080, 108...","[pedestrian2, pedestrian2, pedestrian2, pedest...","[pedestrian, pedestrian, pedestrian, pedestria...","[1183.0, 1185.0, 1186.0, 1185.0, 1184.0, 1186....",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[/work/vita/datasets/JAAD/images/video_0301, /...","[huehue, huehue, huehue, huehue, huehue, huehu...","[00005.png, 00006.png, 00007.png, 00008.png, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[142, 142, 142, 142, 142, 142, 142, 142, 142, ..."
1,0_301_2338b,"[video_0301, video_0301, video_0301, video_030...","[00000, 00001, 00002, 00003, 00004, 00005, 000...","[301, 301, 301, 301, 301, 301, 301, 301, 301, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[1920, 1920, 1920, 1920, 1920, 1920, 1920, 192...","[1080, 1080, 1080, 1080, 1080, 1080, 1080, 108...","[pedestrian1, pedestrian1, pedestrian1, pedest...","[pedestrian, pedestrian, pedestrian, pedestria...","[1090.0, 1091.0, 1091.0, 1092.0, 1092.0, 1093....",...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[/work/vita/datasets/JAAD/images/video_0301, /...","[huehue, huehue, huehue, huehue, huehue, huehu...","[00000.png, 00001.png, 00002.png, 00003.png, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[105, 105, 105, 105, 105, 105, 105, 105, 105, ..."
2,0_302_2344b,"[video_0302, video_0302, video_0302, video_030...","[00010, 00011, 00012, 00013, 00014, 00015, 000...","[302, 302, 302, 302, 302, 302, 302, 302, 302, ...","[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2...","[1920, 1920, 1920, 1920, 1920, 1920, 1920, 192...","[1080, 1080, 1080, 1080, 1080, 1080, 1080, 108...","[pedestrian3, pedestrian3, pedestrian3, pedest...","[pedestrian, pedestrian, pedestrian, pedestria...","[1115.0, 1115.0, 1116.0, 1116.0, 1117.0, 1117....",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[/work/vita/datasets/JAAD/images/video_0302, /...","[huehue, huehue, huehue, huehue, huehue, huehu...","[00010.png, 00011.png, 00012.png, 00013.png, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3..."
3,0_302_2348b,"[video_0302, video_0302, video_0302, video_030...","[00018, 00019, 00020, 00021, 00022, 00023, 000...","[302, 302, 302, 302, 302, 302, 302, 302, 302, ...","[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2...","[1920, 1920, 1920, 1920, 1920, 1920, 1920, 192...","[1080, 1080, 1080, 1080, 1080, 1080, 1080, 108...","[pedestrian2, pedestrian2, pedestrian2, pedest...","[pedestrian, pedestrian, pedestrian, pedestria...","[1142.0, 1142.0, 1142.0, 1142.0, 1142.0, 1143....",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[/work/vita/datasets/JAAD/images/video_03

In [17]:
t[t["video_str"] == "video_0305"]

,video_str,frame_str,video,frame,im_w,im_h,ID,unique_ID,type,x,...,looking,standing_pred,walking_pred,crossing_pred,crossing_true,scenefolderpath,imagefolderpath,filename,will_cross,crossing_at
"video_0305,00000,0_305_2375b",video_0305,00000,305,0,1920,1080,pedestrian1,0_305_2375b,pedestrian,1323.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0305,huehue,00000.png,1,85
"video_0305,00001,0_305_2375b",video_0305,00001,305,1,1920,1080,pedestrian1,0_305_2375b,pedestrian,1322.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0305,huehue,00001.png,1,85
"video_0305,00002,0_305_2375b",video_0305,00002,305,2,1920,1080,pedestrian1,0_305_2375b,pedestrian,1322.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0305,huehue,00002.png,1,85
"video_0305,00003,0_305_2375b",video_0305,00003,305,3,1920,1080,pedestrian1,0_305_2375b,pedestrian,1321.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0305,huehue,00003.png,1,85
"video_0305,00004,0_305_2375b",video_0305,00004,305,4,1920,1080,pedestrian1,0_305_2375b,pedestrian,1323.0,...,0,-1,-1,-1,0,/work/vita/datasets/JAAD/images/video_0305,huehue,00004.png,1,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"video_0305,00085,0_305_2375b",video_0305,00085,305,85,1920,1080,pedestrian1,0_305_2375b,pedestrian,1478.0,...,1,-1,-1,-1,1,/work/vita/datasets/JAAD/images/video_0305,huehue,00085.png,1,85
"video_0305,00000,0_305_2366b",video_0305,00000,305,0,1920,1080,pedestrian3,0_305_2366b,pedestrian,1184.0,...,0,-1,-1,-1,1,/work/vita/datasets/JAAD/images/video_0305,huehue,00000.png,1,0
"video_0305,00004,0_305_2374b",video_0305,00004,305,4,1920,1080,pedestrian4,0_305_2374b,pedestrian,1235.0,...,0,-1,-1,-1,1,/work/vita/datasets/JAAD/images/video_0305,huehue,00004.png,1,4
"video_0305,00000,0_305_2371b",video_0305,00000,305,0,1920,1080,pedestrian2,0_305_2371b,pedestrian,1161.0,...,0,-1,-1,-1,1,/work/vita/datasets/JAAD/images/video_0305,huehue,00000.png,1,0


In [18]:
inv = invert(df_vdf[df_vdf["video"] > 300], 0)
inv[(inv["video"] == 305)& (inv["ID"] == "pedestrian4") ][["frame", "occlusion","crossing_true", "crossing_at",'ID']]

,frame,occlusion,crossing_true,crossing_at,ID
"video_0305,00004,0_305_2374b",4,2,1,4,pedestrian4
"video_0305,00005,0_305_2374b",5,2,1,4,pedestrian4
"video_0305,00006,0_305_2374b",6,2,1,4,pedestrian4
"video_0305,00007,0_305_2374b",7,2,1,4,pedestrian4
"video_0305,00008,0_305_2374b",8,2,1,4,pedestrian4
...,...,...,...,...,...
"video_0305,00145,0_305_2374b",145,0,0,4,pedestrian4
"video_0305,00146,0_305_2374b",146,0,0,4,pedestrian4
"video_0305,00147,0_305_2374b",147,0,0,4,pedestrian4
"video_0305,00148,0_305_2374b",148,0,0,4,pedestrian4


In [19]:
inv = invert(df_vdf[df_vdf["video"] > 300], 0)
inv[(inv["video"] == 331) & (inv["ID"] == "pedestrian4")& (inv["frame"] >= 1)& (inv["frame"] < 177)][["frame", "occlusion","crossing_true", "crossing_at",'ID']]

,frame,occlusion,crossing_true,crossing_at,ID
"video_0331,00005,0_331_2605b",5,2,0,176,pedestrian4
"video_0331,00006,0_331_2605b",6,2,0,176,pedestrian4
"video_0331,00007,0_331_2605b",7,2,0,176,pedestrian4
"video_0331,00008,0_331_2605b",8,2,0,176,pedestrian4
"video_0331,00009,0_331_2605b",9,2,0,176,pedestrian4
...,...,...,...,...,...
"video_0331,00172,0_331_2605b",172,0,0,176,pedestrian4
"video_0331,00173,0_331_2605b",173,0,0,176,pedestrian4
"video_0331,00174,0_331_2605b",174,0,0,176,pedestrian4
"video_0331,00175,0_331_2605b",175,0,0,176,pedestrian4


In [20]:
inv = remove_occlusions(inv)
inv[inv["video"] == 307]["crossing_true"].sum()

304

In [21]:
grouped = get_video_frame_df(inv)
grouped["crossing_true"].apply(lambda x : sum(x)).sum()
grouped[grouped["video"] == 305]["crossing_true"].apply(lambda x : sum(x)).sum()

369

In [22]:
grouped[(grouped["video"] == 305) & (grouped["frame"] < 80) & (grouped["frame"] > 70)]["occlusion"]

3124    [0, 0, 0, 0, 0, 0, 0, 0]
3168    [0, 0, 0, 0, 0, 0, 0, 0]
3212       [0, 0, 0, 0, 0, 0, 0]
3256       [0, 0, 0, 0, 0, 0, 0]
3300       [0, 0, 0, 0, 0, 0, 0]
3344       [0, 0, 0, 0, 0, 0, 0]
3388       [0, 0, 0, 0, 0, 0, 0]
3432       [0, 0, 0, 0, 0, 0, 0]
3476       [0, 0, 0, 0, 0, 0, 0]
Name: occlusion, dtype: object

In [23]:
sum(t["crossing_true"] == 1)

83

In [24]:
def snip(df_vdf, inverted_frames):
    return df_vdf.loc[(df_vdf["will_cross"] == 0) | ((df_vdf["will_cross"] == 1) & (df_vdf["frame"] <= df_vdf["crossing_at"]) & (df_vdf["frame"] >= df_vdf["crossing_at"] - inverted_frames))]

    

In [25]:
def create_inverted_datasets(df_vdf, until=5, fps=30, remove_occlusion=True, infinity=True, do_snip=True):
    """Computes truncated and full datasets for 0 to until seconds inversion."""
    if remove_occlusion:
        df_vdf = remove_occlusions(df_vdf, 0)
        
    suffix = "_luc_occlusion" if not remove_occlusion else "_luc"
        
    for i in range(until):
        inverted_df_vdf = invert(df_vdf, fps*i) 
        t_inverted_df_vdf = truncate(inverted_df_vdf)
        
        inverted_df_vd = get_video_frame_df(inverted_df_vdf)
        t_inverted_df_vd = get_video_frame_df(t_inverted_df_vdf)
        
        df_train, df_val = split_dataframe(inverted_df_vd)
        t_df_train, t_df_val = split_dataframe(t_inverted_df_vd)
        
        save_dataframe(df_train, df_val, i, suffix=suffix)
        save_dataframe(t_df_train, t_df_val, i, suffix=suffix+"_truncated")
        
    if infinity:
        inverted_df_vdf = invert(df_vdf, -1) 
        t_inverted_df_vdf = truncate(inverted_df_vdf)
        
        inverted_df_vd = get_video_frame_df(inverted_df_vdf)
        t_inverted_df_vd = get_video_frame_df(t_inverted_df_vdf)
        
        df_train, df_val = split_dataframe(inverted_df_vd)
        t_df_train, t_df_val = split_dataframe(t_inverted_df_vd)
        
        save_dataframe(df_train, df_val, "infinity", suffix=suffix)
        save_dataframe(t_df_train, t_df_val, "infinity", suffix=suffix+"_truncated")
        
    if do_snip:
        inverted_df_vdf = invert(df_vdf, -1)
        
        for i in range(1, until):
            snip_df_vdf = snip(inverted_df_vdf, fps*i)
            snip_df_vd = get_video_frame_df(snip_df_vdf)
            
            df_train, df_val = split_dataframe(snip_df_vd)
            
            save_dataframe(df_train, df_val, i, suffix="_snip" + suffix)
        

In [26]:
create_inverted_datasets(df_vdf, until=5, fps=30, infinity=True, remove_occlusion=True)


In [27]:
#create_inverted_datasets(df_vdf, until=5, fps=30, infinity=True, remove_occlusion=False)

In [28]:
df = pd.DataFrame()

df = df.append(pd.read_csv(output_folder + "singletxt_val_4s_snip_luc/val.txt"), ignore_index=True)
print(len(df))
df

7590


,video_str,frame_str,im_w,im_h,ID,unique_ID,type,x,y,w,...,looking,standing_pred,walking_pred,crossing_pred,crossing_true,scenefolderpath,imagefolderpath,filename,will_cross,crossing_at
0,"['video_0301', 'video_0301', 'video_0301', 'vi...","['00000', '00000', '00000', '00000', '00000']","[1920, 1920, 1920, 1920, 1920]","[1080, 1080, 1080, 1080, 1080]","['pedestrian1', 'ped4', 'ped3', 'ped2', 'ped5']","['0_301_2338b', '0_301_2332', '0_301_2334', '0...","['pedestrian', 'ped', 'ped', 'ped', 'ped']","[1090.0, 609.0, 1630.0, 1584.0, 636.0]","[647.0, 660.0, 597.0, 602.0, 657.0]","[47.0, 25.0, 84.0, 82.0, 32.0]",...,"[1, 0, 0, 0, 0]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[1, 0, 0, 0, 0]","['/work/vita/datasets/JAAD/images/video_0301',...","['huehue', 'huehue', 'huehue', 'huehue', 'hueh...","['00000.png', '00000.png', '00000.png', '00000...","[1, 0, 0, 0, 0]","[105, -1, -1, -1, -1]"
1,"['video_0302', 'video_0302', 'video_0302']","['00000', '00000', '00000']","[1920, 1920, 1920]","[1080, 1080, 1080]","['ped2', 'pedestrian4', 'ped3']","['0_302_2342', '0_302_2351b', '0_302_2350']","['ped', 'pedestrian', 'ped']","[876.0, 621.0, 903.0]","[659.0, 668.0, 655.0]","[20.0, 29.0, 20.0]",...,"[0, 0, 0]","[-1, -1, -1]","[-1, -1, -1]","[-1, -1, -1]","[0, 0, 0]","['/work/vita/datasets/JAAD/images/video_0302',...","['huehue', 'huehue', 'huehue']","['00000.png', '00000.png', '00000.png']","[0, 0, 0]","[-1, -1, -1]"
2,"['video_0303', 'video_0303', 'video_0303', 'vi...","['00000', '00000', '00000', '00000', '00000']","[1920, 1920, 1920, 1920, 1920]","[1080, 1080, 1080, 1080, 1080]","['pedestrian1', 'ped4', 'ped2', 'ped3', 'ped1']","['0_303_2358b', '0_303_2354', '0_303_2355', '0...","['pedestrian', 'ped', 'ped', 'ped', 'ped']","[1391.0, 737.0, 1289.0, 866.0, 455.0]","[585.0, 652.0, 645.0, 651.0, 637.0]","[51.0, 27.0, 32.0, 30.0, 38.0]",...,"[1, 0, 0, 0, 0]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[1, 0, 0, 0, 0]","['/work/vita/datasets/JAAD/images/video_0303',...","['huehue', 'huehue', 'huehue', 'huehue', 'hueh...","['00000.png', '00000.png', '00000.png', '00000...","[1, 0, 0, 0, 0]","[81, -1, -1, -1, -1]"
3,['video_0304'],['00000'],[1920],[1080],['pedestrian'],['0_304_2359b'],['pedestrian'],[1214.0],[742.0],[54.0],...,[1],[-1],[-1],[-1],[0],['/work/vita/datasets/JAAD/images/video_0304'],['huehue'],['00000.png'],[0],[-1]
4,"['video_0305', 'video_0305', 'video_0305', 'vi...","['00000', '00000', '00000', '00000', '00000']","[1920, 1920, 1920, 1920, 1920]","[1080, 1080, 1080, 1080, 1080]","['pedestrian3', 'pedestrian8', 'pedestrian6', ...","['0_305_2366b', '0_305_2365b', '0_305_2363b', ...","['pedestrian', 'pedestrian', 'pedestrian', 'pe...","[1184.0, 521.0, 571.0, 1100.0, 1161.0]","[630.0, 618.0, 621.0, 629.0, 654.0]","[34.0, 35.0, 30.0, 46.0, 38.0]",...,"[0, 0, 0, 0, 0]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1]","[1, 0, 0, 0, 1]","['/work/vita/datasets/JAAD/images/video_0305',...","['huehue', 'huehue', 'huehue', 'huehue', 'hueh...","['00000.png', '00000.png', '00000.png', '00000...","[1, 0, 0, 0, 1]","[0, -1, -1, -1, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7585,"['video_0313', 'video_0313', 'video_0313']","['00595', '00595', '00595']","[1920, 1920, 1920]","[1080, 1080, 1080]","['people1', 'ped4', 'ped3']","['0_313_65p', '0_313_2462', '0_313_2466']","['ped', 'ped', 'ped']","[1094.0, 91.0, 0.0]","[592.0, 630.0, 631.0]","[78.0, 36.0, 31.0]",...,"[0, 0, 0]","[-1, -1, -1]","[-1, -1, -1]","[-1, -1, -1]","[0, 0, 0]","['/work/vita/datasets/JAAD/images/video_0313',...","['huehue', 'huehue', 'huehue']","['00595.png', '00595.png', '00595.png']","[0, 0, 0]","[-1, -1, -1]"
7586,['video_0313'],['00596'],[1920],[1080],['people1'],['0_313_65p'],['ped'],[1097.0],[591.0],[78.0],...,[0],[-1],[-1],[-1],[0],['/work/vita/datasets/JAAD/images/video_0313'],['huehue'],['00596.png'],[0],[-1]
7587,['video_0313'],['00597'],[1920],[1080],['peopl

In [29]:
df["crossing_true"].apply(lambda x : sum(eval(x))).sum()

2374

In [56]:
for i in range(1, 347):
    video_number = str(f"{i:04d}")
    
    df_original = pd.read_csv("../annotations/converted/"+video_number+".txt").sort_values(by=['frame','ID'])
    df_mine = df_vdf[df_vdf["video_str"] == "video_"+video_number].sort_values(by=['frame','ID'])    
    if len(df_original) == len(df_mine):
        frame_diff = df_original["frame"].to_numpy() - df_mine["frame"].to_numpy()
    
        if sum(frame_diff):
            print(sum(frame_diff))
            print(video_number)
    else:
        print(video_number)
        print(max(df_original["occlusion"]), max(df_mine["occlusion"]))
        print()


0020
1 2

0028
1 2

0039
1 2

0042
1 2

0053
1 2

0061
1 2

0062
1 2

0068
1 2

0070
1 2

0071
1 2

0080
1 2

0082
1 2

0083
1 2

0117
1 2

0118
1 2

0124
1 2

0127
1 1

0129
1 2

0136
1 2

0146
1 2

0150
1 2

0151
1 2

0155
1 2

0157
1 2

0168
1 2

0170
1 2

0185
1 2

0186
1 2

0191
1 2

0194
1 2

0214
1 2

0223
1 2

0232
1 2

0238
1 2

0240
1 2

0242
1 2

0244
1 2

0251
1 2

0263
1 2

0264
1 2

0265
1 2

0268
1 2

0283
1 2

0290
1 2

0291
1 2

0295
1 2

0297
1 2

0298
1 2

0311
1 2

0313
1 2

0314
1 2

0315
1 2

0322
1 2

0323
1 1

0326
1 2

0327
1 2

0330
1 2

0331
1 2

0332
1 2

0339
1 2

0341
1 2

